# ImageNet

We created a supervised machine learning bdp for MNIST, but what if we want to re-use this for ImageNet? Let's quickly write out what would need to happen!

In [1]:
import sys
sys.path.append("../../pybdp")
from src import pybdp
#import pybdp
from pprint import pprint

# Start with an empty project
project = pybdp.load_from_json("../JSON/Supervised Learning V2.json")
print(project)

< Project
Toolbox:

< Toolbox
Blocks: ['Experiment', 'Load Supervised Features', 'Supervised Learning', 'Load Supervised Dataset', 'Cross Validation Split', 'Training Data Preprocessing', 'Testing Data Preprocessing', 'Fit Supervised Model', 'Evaluate Supervised Model', 'Post-processing']
Spaces: ['Model', 'Evaluation Metrics', 'X Train', 'Y Train', 'X Test', 'Y Test', 'X', 'Y'] >

Workbench:

<Workbench
Processors: ['MNIST Experiment', 'Load MNIST', 'Default Supervised Learning', 'Load MNIST Dataset', 'Test-Train Split', 'Image Normalization Preprocessing - Training', 'Image Normalization Preprocessing - Testing', 'Fit Supervised Model - Default', 'Evaluate Supervised Model - Default', 'No Post-processing']
Wires: ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10', 'W11', 'W12', 'W13', 'W14']
Systems: ['MNIST Experiment System', 'Load MNIST System', 'Default Supervised Learning System'] > >


In [2]:
project.copy_add_processor(project.processors_map["Load MNIST Dataset"],
                           {"ID": "Load ImageNet Dataset",
                            "Name": "Load ImageNet Dataset",
                            "Description": "Loads the ImageNet dataset"},
                            copy_wires=True)
print(project)

< Project
Toolbox:

< Toolbox
Blocks: ['Experiment', 'Load Supervised Features', 'Supervised Learning', 'Load Supervised Dataset', 'Cross Validation Split', 'Training Data Preprocessing', 'Testing Data Preprocessing', 'Fit Supervised Model', 'Evaluate Supervised Model', 'Post-processing']
Spaces: ['Model', 'Evaluation Metrics', 'X Train', 'Y Train', 'X Test', 'Y Test', 'X', 'Y'] >

Workbench:

<Workbench
Processors: ['MNIST Experiment', 'Load MNIST', 'Default Supervised Learning', 'Load MNIST Dataset', 'Test-Train Split', 'Image Normalization Preprocessing - Training', 'Image Normalization Preprocessing - Testing', 'Fit Supervised Model - Default', 'Evaluate Supervised Model - Default', 'No Post-processing', 'Load ImageNet Dataset']
Wires: ['W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9', 'W10', 'W11', 'W12', 'W13', 'W14', 'W15', 'W16']
Systems: ['MNIST Experiment System', 'Load MNIST System', 'Default Supervised Learning System'] > >


In [3]:
add_wires = [x.id for x in project.processors_map["Load ImageNet Dataset"].wires]
remove_wires = [x.id for x in project.processors_map["Load MNIST Dataset"].wires]

add_dictionary = {"Wires": add_wires,
                  "Processors": ["Load ImageNet Dataset"]}
remove_dictionary = {"Wires": remove_wires,
                     "Processors": ["Load MNIST Dataset"]}
update_dictionary = {"ID": "Load ImageNet System",
                     "Name": "Load ImageNet System",
                    "Description": "The system representing loading the ImageNet dataset, preparing the cross validation split, and pre-processing the data"}

project.copy_add_system(project.systems_map["Load MNIST System"],
                        update_dictionary,
                        add_dictionary=add_dictionary,
                        remove_dictionary=remove_dictionary,)


project.systems_map["Load ImageNet System"].display_mermaid_graphic()

```mermaid
---
config:
    layout: elk
---
graph LR
subgraph GS0[Load ImageNet System]
subgraph G0[Test-Train Split - Cross Validation Split Block]
direction LR
X0[Test-Train Split]
subgraph G0P[Ports]
direction TB
XX0P0[X]
XX0P1[Y]
end
XX0P0[X] o--o X0
XX0P1[Y] o--o X0
subgraph G0T[Terminals]
direction TB
XX0T0[X Train]
XX0T1[Y Train]
XX0T2[X Test]
XX0T3[Y Test]
end
X0 o--o XX0T0[X Train]
X0 o--o XX0T1[Y Train]
X0 o--o XX0T2[X Test]
X0 o--o XX0T3[Y Test]
end
subgraph G1[Image Normalization Preprocessing - Training - Training Data Preprocessing Block]
direction LR
X1[Image Normalization Preprocessing - Training]
subgraph G1P[Ports]
direction TB
XX1P0[X Train]
XX1P1[Y Train]
end
XX1P0[X Train] o--o X1
XX1P1[Y Train] o--o X1
subgraph G1T[Terminals]
direction TB
XX1T0[X Train]
XX1T1[Y Train]
end
X1 o--o XX1T0[X Train]
X1 o--o XX1T1[Y Train]
end
subgraph G2[Image Normalization Preprocessing - Testing - Testing Data Preprocessing Block]
direction LR
X2[Image Normalization Preprocessing - Testing]
subgraph G2P[Ports]
direction TB
XX2P0[X Train]
XX2P1[Y Train]
XX2P2[X Test]
XX2P3[Y Test]
end
XX2P0[X Train] o--o X2
XX2P1[Y Train] o--o X2
XX2P2[X Test] o--o X2
XX2P3[Y Test] o--o X2
subgraph G2T[Terminals]
direction TB
XX2T0[X Test]
XX2T1[Y Test]
end
X2 o--o XX2T0[X Test]
X2 o--o XX2T1[Y Test]
end
subgraph G3[Load ImageNet Dataset - Load Supervised Dataset Block]
direction LR
X3[Load ImageNet Dataset]
subgraph G3P[Ports]
direction TB
end
subgraph G3T[Terminals]
direction TB
XX3T0[X]
XX3T1[Y]
end
X3 o--o XX3T0[X]
X3 o--o XX3T1[Y]
end
XX0T0[X Train] ---> XX1P0[X Train]
XX0T1[Y Train] ---> XX1P1[Y Train]
XX0T0[X Train] ---> XX2P0[X Train]
XX0T1[Y Train] ---> XX2P1[Y Train]
XX0T2[X Test] ---> XX2P2[X Test]
XX0T3[Y Test] ---> XX2P3[Y Test]
XX3T0[X] ---> XX0P0[X]
XX3T1[Y] ---> XX0P1[Y]
end

```